In [3]:
import pandas as pd
import yfinance as yf
import requests
from bs4 import BeautifulSoup
import xlsxwriter
import numpy as np

# Scrape the list of S&P 500 stocks
res = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = BeautifulSoup(res.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})
tickers = []
for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text.strip()
    tickers.append(ticker)

# Fetch financial data for each stock
data = []
for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        # Fetch the last day's closing price
        closing_price = stock.history(period='1d')['Close'][0]
        pe_ratio = info.get('trailingPE', np.nan)
        pb_ratio = info.get('priceToBook', np.nan)
        ps_ratio = info.get('priceToSalesTrailing12Months', np.nan)
        market_cap = info.get('marketCap', np.nan)
        enterprise_value = info.get('enterpriseValue', np.nan)
        ebitda = info.get('ebitda', np.nan)
        gross_profit = info.get('grossProfits', np.nan)
        ev_to_ebitda = enterprise_value / ebitda if ebitda else np.nan
        ev_to_gp = enterprise_value / gross_profit if gross_profit else np.nan
        data.append([ticker, closing_price, pe_ratio, pb_ratio, ps_ratio, market_cap, enterprise_value,
                     ebitda, gross_profit, ev_to_ebitda, ev_to_gp])
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")

# Create DataFrame from data
df = pd.DataFrame(data, columns=['Symbol', 'Price', 'P/E Ratio', 'P/B Ratio', 'P/S Ratio', 'Market Cap',
                                 'Enterprise Value', 'EBITDA', 'Gross Profit', 'EV/EBITDA', 'EV/GP'])

# Filter top 50 stocks by Market Cap
df_top_50 = df.nlargest(50, 'Market Cap')

# Save the DataFrame to an Excel file with formatting
writer = pd.ExcelWriter('top_50_stocks.xlsx', engine='xlsxwriter')
df_top_50.to_excel(writer, sheet_name='Sheet1', index=False)

# Access the XlsxWriter workbook and worksheet objects from the DataFrame writer object.
workbook = writer.book
worksheet = writer.sheets['Sheet1']

# Create a format for the header cells
header_format = workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'valign': 'top',
    'fg_color': '#D7E4BC',
    'border': 1})

# Create a format for float cells
float_format = workbook.add_format({'num_format': '#.###'})

# Create a format for percent cells
percent_format = workbook.add_format({'num_format': '0.00%'})

# Write the column headers with the defined format.
for col_num, value in enumerate(df_top_50.columns.values):
    worksheet.write(0, col_num, value, header_format)

# Apply the float format to columns with float values
for col_num in range(1, df_top_50.shape[1]):  # column numbers are zero-indexed
    worksheet.set_column(col_num, col_num, cell_format=float_format)
    
# Apply the percent format to specific columns (e.g., 'P/E Ratio', 'P/B Ratio', etc.)
# We assume here that 'P/E Ratio', 'P/B Ratio', 'P/S Ratio' are in columns 3, 4, 5 respectively (Excel columns 'C', 'D', 'E')
for column in ['C', 'D', 'E']:  
    worksheet.set_column(f'{column}:{column}', 18, percent_format)

writer.save()




BRK.B: No data found, symbol may be delisted
Error fetching data for BRK.B: index 0 is out of bounds for axis 0 with size 0
BF.B: Period '1d' is invalid, must be one of ['1mo', '3mo', '6mo', 'ytd', '1y', '2y', '5y', '10y', 'max']
Error fetching data for BF.B: index 0 is out of bounds for axis 0 with size 0


/var/folders/ck/5n88btq11tg1yg8ll_xpw7nr0000gn/T/ipykernel_54407/2192111785.py:67: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [4]:
df_top_50

,Symbol,Price,P/E Ratio,P/B Ratio,P/S Ratio,Market Cap,Enterprise Value,EBITDA,Gross Profit,EV/EBITDA,EV/GP
44,AAPL,172.570007,29.199663,43.655453,7.048396,2714301890560,2.786605e+12,1.237880e+11,1.707820e+11,22.511106,16.316736
317,MSFT,308.970001,32.869152,11.802659,11.066681,2297343246336,2.297070e+12,1.000800e+11,1.356200e+11,22.952339,16.937547
22,GOOGL,117.510002,25.16274,5.730238,5.048712,1436924116992,1.333281e+12,8.749600e+10,1.566330e+11,15.238191,8.512132
23,GOOG,117.919998,24.773108,5.750232,5.042421,1435133542400,1.340010e+12,8.749600e+10,1.566330e+11,15.315102,8.555095
25,AMZN,110.260002,250.59091,7.319437,2.155302,1131311792128,1.244730e+12,5.719400e+10,2.251520e+11,21.763303,5.528400
348,NVDA,283.399994,165.73099,31.622406,25.983728,700885041152,7.046116e+11,7.121000e+09,1.535600e+10,98.948409,45.885101
311,META,233.809998,29.008682,4.807542,5.112554,599937777664,5.881446e+11,4.245600e+10,9.285500e+10,13.853038,6.334011
443,TSLA,167.979996,49.40588,11.077552,6.188326,532412596224,5.204747e+11,1.666600e+10,2.085300e+10,31.229728,24.959222
474,V,231.380005,30.891857,12.936376,15.640909,484602281984,4.841176e+11,2.169300e+10,2.856700e+10,22.316765,16.946742
463,UNH,491.230011,22.471638,5.630466,1.362251,457640181760,4.896719e+11,3.215300e+10,7.961700e+10,15.229430,6.150343
